In [1]:
import os
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import TensorBoard
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam,RMSprop
from tensorflow.keras.callbacks import LearningRateScheduler


import matplotlib.pyplot as plt
import pickle
import seaborn as sns
from sklearn.metrics import confusion_matrix
import plotly.express as px

import keras
from keras.models import Sequential
from keras.layers import Conv2D,Add,MaxPooling2D, Dense, BatchNormalization,Input,Flatten, Dropout,GlobalMaxPooling2D,Lambda
from keras.callbacks import EarlyStopping

In [2]:
os.listdir('C:\\Users\\josek\\Documents\\gitMCC\\MCC-2023-1AD\\CNN-aves\\kaggle\\input\\100-bird-species')

['birds.csv',
 'EfficientNetB0-525-(224 X 224)- 98.97.h5',
 'test',
 'train',
 'valid']

In [3]:
train = 'C:\\Users\\josek\\Documents\\gitMCC\\MCC-2023-1AD\\CNN-aves\\kaggle\\input\\100-bird-species\\train'
validation = 'C:\\Users\\josek\\Documents\\gitMCC\\MCC-2023-1AD\\CNN-aves\\kaggle\\input\\100-bird-species\\valid'
test = 'C:\\Users\\josek\\Documents\\gitMCC\\MCC-2023-1AD\\CNN-aves\\kaggle\\input\\100-bird-species\\test'

In [4]:
train_datagen = ImageDataGenerator(rescale = 1./255)#initialize train generator 
                                 
valid_datagen = ImageDataGenerator(rescale = 1.0/255.) #initialize validation generator 

test_datagen = ImageDataGenerator(rescale = 1.0/255.) #initialize validation generator

In [5]:
train_generator = train_datagen.flow_from_directory(train, target_size=(224,224),batch_size=32,class_mode='categorical')

validation_generator = valid_datagen.flow_from_directory(validation, target_size=(224,224),batch_size=32,class_mode='categorical')

test_generator = test_datagen.flow_from_directory(test, target_size=(224,224),batch_size=32,class_mode='categorical')

Found 811 images belonging to 5 classes.
Found 25 images belonging to 5 classes.
Found 25 images belonging to 5 classes.


In [6]:
from tensorflow.keras.applications import InceptionV3
from tensorflow.keras.models import Model

#Load Model
inception = tf.keras.applications.InceptionV3(weights='imagenet',include_top=False,input_shape=(224,224,3))


inception.trainable = True
for layer in inception.layers[:197]:
    layer.trainable = False 

In [7]:
#get the last layer    
last_layer = inception.get_layer('mixed7')

print('last layer output shape: ', last_layer.output_shape)

layer_output = last_layer.output

last layer output shape:  (None, 12, 12, 768)


In [8]:
n_categories = len(os.listdir(train))# number of categories
print(n_categories)

5


In [9]:
# x  = BatchNormalization()(layer_output)
x = Flatten()(layer_output)
x = Dense(1024,activation='relu',kernel_regularizer=tf.keras.regularizers.l2(0.001))(x)
x = Dropout(0.4)(x)
x = Dense(n_categories, activation='softmax')(x)

model = Model(inputs=inception.inputs, outputs=x)

In [10]:
# Set the training parameters
model.compile(optimizer = RMSprop(learning_rate=0.0001), 
              loss = 'categorical_crossentropy', 
              metrics = ['accuracy'])

In [11]:
def scheduler(epoch, lr):
    if epoch < 10:
        return lr
    else:
        return lr * tf.math.exp(-0.1)
    
callback = tf.keras.callbacks.LearningRateScheduler(scheduler)

In [12]:
history = model.fit(
        train_generator,
        validation_data=validation_generator,
        epochs=25,
        callbacks=[callback],
        batch_size=2
    )

    #history = model.fit(
     #       train_generator,
     #       validation_data = validation_generator,
     #       epochs = 25,
    #callbacks=[callback])

Epoch 1/25


ResourceExhaustedError: Graph execution error:

Detected at node 'gradient_tape/model/dense/MatMul/MatMul_1' defined at (most recent call last):
    File "c:\Users\josek\.conda\envs\tf\lib\runpy.py", line 197, in _run_module_as_main
      return _run_code(code, main_globals, None,
    File "c:\Users\josek\.conda\envs\tf\lib\runpy.py", line 87, in _run_code
      exec(code, run_globals)
    File "C:\Users\josek\AppData\Roaming\Python\Python39\site-packages\ipykernel_launcher.py", line 18, in <module>
      app.launch_new_instance()
    File "C:\Users\josek\AppData\Roaming\Python\Python39\site-packages\traitlets\config\application.py", line 1075, in launch_instance
      app.start()
    File "C:\Users\josek\AppData\Roaming\Python\Python39\site-packages\ipykernel\kernelapp.py", line 739, in start
      self.io_loop.start()
    File "C:\Users\josek\AppData\Roaming\Python\Python39\site-packages\tornado\platform\asyncio.py", line 205, in start
      self.asyncio_loop.run_forever()
    File "

Evaluacion

In [ ]:
results = pd.DataFrame(history.history)
results.tail()

In [ ]:
fig = px.line(results,y=[results['accuracy'],results['val_accuracy']],template="seaborn",color_discrete_sequence=['#7F00FF','#00bfff'])
fig.update_layout(   
    title_font_color="#41BEE9", 
    xaxis=dict(color="#41BEE9",title='Epochs'), 
    yaxis=dict(color="#41BEE9")
 )
fig.show()

In [ ]:
fig = px.line(results,y=[results['loss'],results['val_loss']],template="seaborn",color_discrete_sequence=['#7F00FF','#00bfff'])
fig.update_layout(   
    title_font_color="#41BEE9", 
    xaxis=dict(color="#41BEE9",title='Epochs'), 
    yaxis=dict(color="#41BEE9")
 )
fig.show()

In [ ]:
model.save('model.h5')

In [ ]:
model.evaluate(test_generator,verbose=0)